In [ ]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

[W 2024-02-20 16:35:53.527 ServerApp] A `_jupyter_server_extension_points` function was not found in notebook_shim. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyter Server.
[I 2024-02-20 16:35:53.528 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2024-02-20 16:35:53.532 ServerApp] jupyter_server_terminals | extension was successfully linked.
[W 2024-02-20 16:35:53.534 LabApp] 'iopub_data_rate_limit' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2024-02-20 16:35:53.538 ServerApp] ServerApp.iopub_data_rate_limit config is deprecated in 2.0. Use ZMQChannelsWebsocketConnection.iopub_data_rate_limit.
[I 2024-02-20 16:35:53.538 ServerApp] jupyterlab | extension was successfully linked.
[I 2024-02-20 16:35:53.542 ServerApp] notebook | extension was successfully linked.
[I 2024-

In [84]:

import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_iris
from scipy.stats import chi2_contingency
import re
from datetime import datetime
from collections import Counter
from sklearn.preprocessing import MinMaxScaler


df = pd.read_pickle(r'/Users/maya/Documents/Information Systems/שנה ג׳/למידת מכונה/פרוייקט קורס/XY_train.pkl')
print(df)

           textID                                               text  \
0      b8f4e560fa   but yeah i like purple maybe thats why!! ;)  ...   
1      f81a1511b2  _Henrie haha i WISH i coudl meet you.. you sho...   
2      3e9e3f0d69          nah, you`re just altered forever   Enjoy.   
3      a068b95bd9   Can you ask Ryan why he stopped following me ...   
4      e6da2d1835  New issue of  in the office...desperately want...   
...           ...                                                ...   
12267  35ee036565  2.5 years here & left Dell OKC for the last ti...   
12268  10c23a0f46   That is the best sticker EVER. Also. I dig yo...   
12269  a2dd416423  getting dinner ready not much going on in my l...   
12270  d9f7bd1045                                 Ow. Sunburn hurts!   
12271  33b4f870e8        dont ya know? people love the human society   

      sentiment        message_date account_creation_date  \
0      positive   2023-10-2 7:41:52      2015-4-13 0:1:16   
1      positi

In [85]:

#------------PRE-PROCESSING---------------#

#--Missing values--
# Calculate missing values before deletion
missing_values_before = df.isnull().sum()
df = df.dropna(thresh=df.shape[1]-2)
missing_values_after = df.isnull().sum()
missing_values_table = pd.DataFrame({
    'Before': missing_values_before,
    'After': missing_values_after,
    'Difference': missing_values_before - missing_values_after
})
print("Missing Values Before and After Deletion:")
print(missing_values_table)

#--filling email iwith the value unknowm--
df['email'] = df['email'].fillna('unknown')
missing_values_count = df['email'].isnull().sum()

#--Filling embedded_content and platform according to their probability--
embedded_content_prob = df['embedded_content'].value_counts(normalize=True)
platform_prob = df['platform'].value_counts(normalize=True)

def impute_missing_values(row, prob_dist):
    if pd.isnull(row):
        return np.random.choice(prob_dist.index, p=prob_dist.values)
    else:
        return row
df['embedded_content'] = df['embedded_content'].apply(lambda x: impute_missing_values(x, embedded_content_prob))
df['platform'] = df['platform'].apply(lambda x: impute_missing_values(x, platform_prob))

#--Fill missing values for email_verified and blue_tick--
df['email_verified'].fillna(df['blue_tick'], inplace=True)
df['blue_tick'].fillna(df['email_verified'], inplace=True)

#--filling gender randomly--
df['gender'].replace('None', np.nan, inplace=True)
gender_counts = df['gender'].value_counts()
df['gender'].fillna(pd.Series(np.random.choice(gender_counts.index,
                                               size=len(df.index),
                                               p=(gender_counts / gender_counts.sum()))),
                    inplace=True)

#--Droping the rows with missing values--
df = df.dropna()

#Converting message_date to categorical
df['message_date'] = pd.to_datetime(df['message_date'])
df['hour'] = df['message_date'].dt.hour
morning_interval = range(6, 12)  # 6:00 AM to 11:59 AM
noon_interval = range(12, 18)     # 12:00 PM to 5:59 PM
evening_interval = range(18, 24)  # 6:00 PM to 11:59 PM
def categorize_hour(hour):
    if hour in morning_interval:
        return 'Morning'
    elif hour in noon_interval:
        return 'Noon'
    elif hour in evening_interval:
        return 'Evening'
    else:
        return 'Night'
df['message_time_category'] = df['hour'].apply(categorize_hour)
df.drop(columns=['hour'], inplace=True)



Missing Values Before and After Deletion:
                         Before  After  Difference
textID                        0      0           0
text                          0      0           0
sentiment                     0      0           0
message_date                  0      0           0
account_creation_date         0      0           0
previous_messages_dates       0      0           0
date_of_new_follower          0      0           0
date_of_new_follow            0      0           0
email                       891    825          66
gender                     1619   1511         108
email_verified             1023    941          82
blue_tick                  1439   1354          85
embedded_content            906    854          52
platform                    750    697          53


/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_965/54234375.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['email_verified'].fillna(df['blue_tick'], inplace=True)
/var/folders/m3/b_36s2ds2f304ldssyhqv2sm0000gn/T/ipykernel_965/54234375.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

In [86]:
#Changes in Data For n-gram#

#Lower case
lower = df['text'].apply(str.lower)


#Stemming
from nltk.stem import SnowballStemmer
stem = SnowballStemmer('english')
stemmed = lower.apply(lambda x: ' '.join(stem.stem(word) for word in str(x).split()))
    

#removing punctuation
import re
rem_punc = stemmed.apply(lambda x: re.sub(r'[^\w\s]',' ',x))


#removing numbers
rem_num = rem_punc.apply(lambda x: " ".join(x for x in x.split() if not x.isdigit()))

#removing words with 1 letter
rem_length1 = rem_num.apply(lambda x: re.sub(r'\b\w{1}\b',' ',x))

# Remove top 0.05% of most common or not common words
h_pct = 0.05
l_pct = 0.05

#removing the top $h_pct of the most frequent words 
high_freq = pd.Series(''.join(rem_length1).split()).value_counts()[:int(pd.Series(' '.join(rem_length1).split()).count()*h_pct/100)]

rem_high = rem_length1.apply(lambda x: " ".join(x for x in x.split() if x not in high_freq))
    
#removing the top $l_pct of the least frequent words
low_freq = pd.Series(''.join(rem_high).split()).value_counts()[:-int(pd.Series(' '.join(rem_high).split()).count()*l_pct/100):-1]

rem_low = rem_high.apply(lambda x: " ".join(x for x in x.split() if x not in low_freq))

# Remove double spaces
rem_punc = rem_low.apply(lambda x: re.sub(r'[^\w\s]',' ',x))

df['clean_text'] = rem_punc



In [87]:

#1.Create a new column based on the length of messages
df['message_length'] = df['text'].apply(lambda x: len(x))
#2.Creating the number of messages sent by the user
df['num_messages_sent'] = df['previous_messages_dates'].apply(len)
#3.creating num of followers and following
df['follower_count'] = df['date_of_new_follower'].apply(lambda x: len(x))
df['following_count'] = df['date_of_new_follow'].apply(lambda x: len(x))

#4.creating the difference between followers and following
df['follower_following_diff'] = abs(df['following_count'] - df['follower_count'])
new_columns_df = df[['follower_count', 'following_count', 'follower_following_diff']]

# #5.N-GRAM

#------- tamar ngram ---------#
# ngram_range = (2, 2)  # Set N-gram range to bi-grams
# vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=20)
# X_text_features = vectorizer.fit_transform(df['text'])
# df = pd.concat([df.reset_index(drop=True), pd.DataFrame(X_text_features.toarray())], axis=1)
#------- tamar ngram ---------#

X = df['clean_text']
Y = df['sentiment']

ngram_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features = 100)
X_ngrams = ngram_vectorizer.fit_transform(X).toarray()

# Check words frequency
# sum_of_words = X_ngrams.sum(axis = 0)
# words_freq = [(word, sum_of_words[i]) for word, i in ngram_vectorizer.vocabulary_.items()]
# words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
# 
# print(words_freq)

df_output = pd.DataFrame(data = X_ngrams, columns = ngram_vectorizer.get_feature_names_out())
# df = pd.concat([df.reset_index(drop=True), pd.DataFrame(df_output)], axis=1)



#6.Extracting email domain endings
def extract_email_domain_ending(email):
    if pd.isnull(email):
        return 'Missing'
    match = re.search(r'\.(\w+)$', email)
    if match:
        return match.group(1)
    else:
        return 'Unknown'
df['email_domain_ending'] = df['email'].apply(extract_email_domain_ending)
email_domain_ending_counts = df['email_domain_ending'].value_counts()

#7.Creating seniority in years
df['account_creation_date'] = pd.to_datetime(df['account_creation_date'])
current_date = datetime.now()
df['seniority'] = (current_date - df['account_creation_date']).dt.days / 365.25

#8.Creating the average time difference between messages
def calculate_average_time_difference(message_dates_array):
    message_dates = [datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S') for date_str in message_dates_array]
    time_diffs = []
    for i in range(1, len(message_dates)):
        time_diff = (message_dates[i] - message_dates[i - 1]).total_seconds()
        if time_diff < 0:
            time_diff = (message_dates[i - 1] - message_dates[i]).total_seconds()
        time_diffs.append(time_diff)
    if len(time_diffs) > 0:
        average_time_difference = sum(time_diffs) / len(time_diffs)
        average_time_difference = int(average_time_difference)
        return average_time_difference
    else:
        return None
df['average_time_difference'] = df['previous_messages_dates'].apply(calculate_average_time_difference)

#4.Feature representation--------------------------------------------------->

#1.Normalized values by dividing each column by its maximum value
columns_to_normalize = ['message_length', 'num_messages_sent', 'follower_count', 'following_count',
                        'seniority', 'average_time_difference', 'follower_following_diff']
for column in columns_to_normalize:
    max_value = df[column].max()
    df[f'normalized_{column}'] = df[column] / max_value
for column in columns_to_normalize:
    normalized_column_name = f'normalized_{column}'
    print(f"Normalized {column}:")
    print(df[normalized_column_name])

#2.One-hot encoding
email_domain_ending_onehot = pd.get_dummies(df['email_domain_ending'], prefix='email_ending')
embedded_content_onehot = pd.get_dummies(df['embedded_content'], prefix='embedded_content')
platform_onehot = pd.get_dummies(df['platform'], prefix='platform')
message_time_category_onehot = pd.get_dummies(df['message_time_category'], prefix='message_time')
df = pd.concat([df, email_domain_ending_onehot, embedded_content_onehot, platform_onehot, message_time_category_onehot], axis=1)
df.drop(['email_domain_ending', 'embedded_content', 'platform', 'message_time_category'], axis=1, inplace=True)

#3.Converting to binary values
bool_to_binary = {True: 1, False: 0}
df['email_verified'] = df['email_verified'].map(bool_to_binary)
df['blue_tick'] = df['blue_tick'].map(bool_to_binary)

for column in df.columns:
    if df[column].dtype == bool:
        df[column] = df[column].astype(int) 

gender_to_binary = {'F': 1, 'M': 0}
df['gender'] = df['gender'].map(gender_to_binary)

#4. Normalization the n-gram features
scaler = MinMaxScaler()
X_ngrams_normalized = scaler.fit_transform(X_ngrams)
df_normalized = pd.DataFrame(X_ngrams_normalized, columns=ngram_vectorizer.get_feature_names_out())
df = pd.concat([df.reset_index(drop=True), df_normalized], axis=1)


#6.Data arrangement
sentiment_mapping = {'positive': 1, 'negative': -1}
df['sentiment'] = df['sentiment'].map(sentiment_mapping)
df = df.drop(columns=['text', 'previous_messages_dates', 'message_date',
                      'email', 'date_of_new_follower', 'date_of_new_follow',
                      'account_creation_date', 'message_length', 'num_messages_sent',
                      'follower_count', 'following_count', 'follower_following_diff',
                      'seniority', 'average_time_difference', 'clean_text'])


#5.Feature selection-------------------------------------------------------->

df.head()

#6.dimensionality reduction------------------------------------------------->

Normalized message_length:
0        0.361702
1        0.843972
2        0.297872
3        0.397163
4        0.751773
           ...   
12266    0.595745
12267    0.950355
12268    0.687943
12269    0.411348
12270    0.127660
Name: normalized_message_length, Length: 12026, dtype: float64
Normalized num_messages_sent:
0        0.244898
1        0.979592
2        0.612245
3        0.102041
4        0.163265
           ...   
12266    0.714286
12267    0.448980
12268    0.469388
12269    0.061224
12270    0.142857
Name: normalized_num_messages_sent, Length: 12026, dtype: float64
Normalized follower_count:
0        0.897959
1        0.367347
2        0.306122
3        0.020408
4        0.040816
           ...   
12266    0.469388
12267    0.489796
12268    0.775510
12269    0.122449
12270    0.163265
Name: normalized_follower_count, Length: 12026, dtype: float64
Normalized following_count:
0        0.285714
1        0.857143
2        0.489796
3        0.081633
4        0.122449
           .

,textID,sentiment,gender,email_verified,blue_tick,normalized_message_length,normalized_num_messages_sent,normalized_follower_count,normalized_following_count,normalized_seniority,...,week,weekend,well,were,when,whi,who,wish,would,yeah
0,b8f4e560fa,1,1,0,0,0.361702,0.244898,0.897959,0.285714,0.795477,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5
1,f81a1511b2,1,0,1,1,0.843972,0.979592,0.367347,0.857143,0.955261,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0
2,3e9e3f0d69,1,1,1,1,0.297872,0.612245,0.306122,0.489796,0.946165,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,a068b95bd9,-1,0,0,0,0.397163,0.102041,0.020408,0.081633,0.749508,...,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0
4,e6da2d1835,-1,0,0,0,0.751773,0.163265,0.040816,0.122449,0.914208,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [74]:
# # Separate the dataset into positive and negative sentiment groups
# positive_sentiment = df[df['sentiment'] == 1]
# negative_sentiment = df[df['sentiment'] == -1]
# 
# # Remove 'sentiment', 'textID' columns as we are not considering it as a feature
# features = df.drop(columns=['sentiment', 'textID']).columns
# 
# # print(features)
# 
# fisher_scores = {}
# 
# # Calculate Fisher score for each feature
# for feature in features:
#     mean_positive = positive_sentiment[feature].mean()
#     mean_negative = negative_sentiment[feature].mean()
#     var_positive = positive_sentiment[feature].var()
#     var_negative = negative_sentiment[feature].var()
#     
#     # print(var_negative)
#     # print(var_positive)
#     
#     fisher_score = ((mean_positive - mean_negative)**2) / (var_positive + var_negative)
#     # print(feature)
#     fisher_scores[feature] = round(fisher_score, 3)  # Round to 3 decimal places
#     
# # print(fisher_scores)
# 
# # Create DataFrame from dictionary
# fisher_df = pd.DataFrame.from_dict(fisher_scores, orient='index', columns=['Fisher Score'])
# 
# # Sort features based on their Fisher scores
# fisher_df = fisher_df.sort_values(by='Fisher Score', ascending=False)
# 
# # Set options to display all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# 
# # Print the DataFrame
# fisher_df


,Fisher Score
normalized_num_messages_sent,2.077
normalized_follower_count,1.861
normalized_following_count,1.762
normalized_follower_following_diff,1.258
email_verified,0.202
blue_tick,0.169
gender,0.150
platform_instagram,0.129
normalized_seniority,0.104
thank,0.075


In [1]:
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np

# Assuming 'X_cat' is your feature matrix and 'Y' is your target variable
# df = df.drop(columns=['normalized_average_time_difference', 'normalized_follower_following_diff'])

# Extract the target variable 'sentiment'
Y = df['sentiment']

# Drop the 'textID' column from the feature DataFrame
X_cat = df.drop(columns=['textID', 'sentiment'])

# Initialize SelectKBest with chi-squared as the scoring function and k=15
chi2_features = SelectKBest(chi2, k=15)

# Fit SelectKBest to the data and transform it
X_cat_kbest = chi2_features.fit_transform(X_cat, Y)

# Get the p-values of the features and round to 4 digits
p_values = chi2_features.pvalues_
p_values_rounded = np.round(p_values, 6)

# Get the selected features
selected_features = X_cat.columns[chi2_features.get_support()]

# Convert the array of rounded p-values into a DataFrame
p_values_df = pd.DataFrame({'Feature': X_cat.columns, 'P-value': p_values_rounded})

# Filter the selected features based on the support
selected_features_df = pd.DataFrame({'Feature': selected_features})

# Print the DataFrames
print("P-values DataFrame:")
print(p_values_df)

print("\nSelected Features DataFrame:")
selected_features_df




NameError: name 'df' is not defined

In [91]:
p_values_df_sorted = p_values_df.sort_values(by='P-value')
selected_features_with_p_values = pd.merge(selected_features_df, p_values_df_sorted, on='Feature')

selected_features_with_p_values


,Feature,P-value
0,gender,0.0
1,email_verified,0.0
2,blue_tick,0.0
3,normalized_num_messages_sent,0.0
4,normalized_follower_count,0.0
5,normalized_following_count,0.0
6,email_ending_edu,0.0
7,embedded_content_False,0.0
8,embedded_content_link,0.0
9,embedded_content_mp4,0.0


In [42]:
# Create a DataFrame with the selected features
selected_df = pd.DataFrame(X_cat_kbest, columns=selected_features)

# Add 'sentiment' and 'textID' columns to the DataFrame
selected_df['sentiment'] = df['sentiment']
selected_df['textID'] = df['textID']

# Reorder the columns to have 'sentiment' and 'textID' as the first columns
selected_df = selected_df[['textID', 'sentiment'] + selected_features.tolist()]

# Print the DataFrame
selected_df

,textID,sentiment,gender,email_verified,blue_tick,normalized_num_messages_sent,normalized_follower_count,normalized_following_count,normalized_follower_following_diff,embedded_content_mp4,platform_instagram,message_time_Evening
0,b8f4e560fa,1,1.0,0.0,0.0,0.244898,0.897959,0.285714,0.612245,1.0,0.0,0.0
1,f81a1511b2,1,0.0,1.0,1.0,0.979592,0.367347,0.857143,0.489796,0.0,0.0,0.0
2,3e9e3f0d69,1,1.0,1.0,1.0,0.612245,0.306122,0.489796,0.183673,0.0,0.0,0.0
3,a068b95bd9,-1,0.0,0.0,0.0,0.102041,0.020408,0.081633,0.061224,1.0,0.0,0.0
4,e6da2d1835,-1,0.0,0.0,0.0,0.163265,0.040816,0.122449,0.081633,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12021,d44a6cf5cd,1,1.0,1.0,1.0,0.714286,0.469388,0.653061,0.183673,0.0,1.0,0.0
12022,35ee036565,1,1.0,0.0,0.0,0.448980,0.489796,0.244898,0.244898,0.0,0.0,0.0
12023,10c23a0f46,1,1.0,1.0,1.0,0.469388,0.775510,0.306122,0.469388,0.0,0.0,0.0
12024,a2dd416423,-1,1.0,1.0,0.0,0.061224,0.122449,0.102041,0.020408,0.0,0.0,1.0


In [75]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Assuming 'X_cat' is your feature matrix and 'Y' is your target variable
# df = df.drop(columns=['normalized_average_time_difference', 'normalized_follower_following_diff'])

# Extract the target variable 'sentiment'
Y = df['sentiment']

# Drop the 'textID' column from the feature DataFrame
X_cat = df.drop(columns=['textID', 'sentiment'])

# Initialize Logistic Regression classifier
classifier = LogisticRegression()

# Initialize RFE with the classifier and the number of desired features
num_features_to_select = 15  # Change this value as needed
rfe = RFE(classifier, n_features_to_select=num_features_to_select)

# Fit RFE to the data
rfe.fit(X_cat, Y)

# Get the ranking of features
feature_ranking = rfe.ranking_

# Get the selected features
selected_features = X_cat.columns[rfe.support_]

# Print the ranking of features and selected features
print("Feature Ranking:")
for feature, rank in zip(X_cat.columns, feature_ranking):
    print(f"{feature}: {rank}")

print("\nSelected Features:")
print(selected_features)


Feature Ranking:
gender: 10
email_verified: 5
blue_tick: 18
normalized_message_length: 72
normalized_num_messages_sent: 1
normalized_follower_count: 1
normalized_following_count: 1
normalized_seniority: 1
email_ending_Unknown: 97
email_ending_com: 102
email_ending_de: 34
email_ending_edu: 27
email_ending_gov: 90
email_ending_il: 87
email_ending_jp: 108
email_ending_ke: 62
email_ending_org: 88
email_ending_ru: 89
embedded_content_False: 19
embedded_content_jpeg: 20
embedded_content_link: 98
embedded_content_mp4: 84
platform_facebook: 110
platform_instagram: 8
platform_telegram: 47
platform_tiktok: 96
platform_whatsapp: 79
platform_x: 31
message_time_Evening: 13
message_time_Morning: 116
message_time_Night: 111
message_time_Noon: 60
after: 93
again: 12
an: 28
as: 106
awesom: 1
bad: 1
bed: 114
been: 50
best: 1
better: 3
bit: 63
by: 15
cant: 24
come: 115
cool: 2
could: 57
did: 91
didn: 26
dont: 101
down: 4
enjoy: 1
even: 49
ever: 37
final: 6
follow: 94
friend: 32
fun: 1
girl: 66
gonna: 22


In [69]:
# from sklearn.feature_selection import chi2, SelectKBest
# 
# # Assuming 'df' is your DataFrame and 'Y' is your target variable
# 
# 
# # Extract the target variable 'sentiment'
# Y = df['sentiment']
# 
# # Drop the 'textID' column from the feature DataFrame
# X_cat = df.drop(columns=['textID', 'sentiment'])
# 
# # Initialize SelectKBest with chi-squared as the scoring function
# chi2_features = SelectKBest(score_func=chi2, k = 15)
# 
# # Fit SelectKBest to the data and transform it
# X_cat_kbest = chi2_features.fit_transform(X_cat, Y)
# # 
# 
# 
# print('orginial', X_cat.shape[1])
# print('reduced', X_cat_kbest.shape[1])

orginial 134
reduced 15


In [25]:
# # Assuming your dictionary is named 'feature_scores' and your DataFrame is named 'df'
# 
# # Filter keys based on score threshold
# selected_features = [key for key, score in fisher_scores.items() if score >= 0.02]
# 
# # Drop columns from DataFrame that are not in selected features
# df = df[['textID', 'sentiment'] + selected_features]
# 
# df


NameError: name 'fisher_scores' is not defined

In [32]:
# # Check for NaN values in each column
# nan_columns = df.columns[df.isna().any()].tolist()
# print("Columns with NaN values:")
# print(nan_columns)
# 
# # Check for NaN values in each row
# nan_rows = df[df.isna().any(axis=1)]
# print("Rows with NaN values:")
# print(nan_rows)

Columns with NaN values:
[]
Rows with NaN values:
Empty DataFrame
Columns: [textID, sentiment, gender, email_verified, blue_tick, normalized_message_length, normalized_num_messages_sent, normalized_follower_count, normalized_following_count, normalized_seniority, normalized_average_time_difference, normalized_follower_following_diff, email_ending_Unknown, email_ending_com, email_ending_de, email_ending_edu, email_ending_gov, email_ending_il, email_ending_jp, email_ending_ke, email_ending_org, email_ending_ru, embedded_content_False, embedded_content_jpeg, embedded_content_link, embedded_content_mp4, platform_facebook, platform_instagram, platform_telegram, platform_tiktok, platform_whatsapp, platform_x, message_time_Evening, message_time_Morning, message_time_Night, message_time_Noon, after, again, an, as, awesom, bad, bed, been, best, better, bit, by, cant, come, cool, could, did, didn, dont, down, enjoy, even, ever, final, follow, friend, fun, girl, gonna, guy, haha, has, hate, he, h

In [26]:
# from sklearn.feature_selection import RFE
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# import pandas as pd
# 
# # Assuming 'df' is your DataFrame with features and target variable
# 
# # Splitting the data into features (X) and target variable (y)
# X = df.drop(columns=['sentiment', 'textID'])
# y = df['sentiment']
# 
# # Splitting the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 
# # Initialize the logistic regression model
# model = LogisticRegression()
# 
# # Initialize RFE with the logistic regression model and number of features to select
# rfe = RFE(estimator=model, n_features_to_select=5, step=1)
# 
# # Fit RFE to the training data
# rfe.fit(X_train, y_train)
# 
# # Get the selected features
# selected_features = X.columns[rfe.support_]
# 
# # Train the model using only selected features
# model.fit(X_train[selected_features], y_train)
# 
# # Make predictions on the testing set
# y_pred = model.predict(X_test[selected_features])
# 
# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)


Accuracy: 0.9858686616791354


In [43]:
# from sklearn.feature_selection import RFE
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score
# 
# # Assuming 'df' is your DataFrame and 'target_column' is your target variable
# 
# # Splitting the data into features (X) and target variable (y)
# X = selected_df.drop(columns=['sentiment', 'textID', 'normalized_follower_following_diff'])
# y = selected_df['sentiment']
# 
# # Splitting the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 
# # Initialize the logistic regression model
# model = LogisticRegression()
# 
# # Initialize RFE with the logistic regression model and desired number of features to select
# rfe = RFE(estimator=model, n_features_to_select=24)  # Change the number of features as desired
# 
# # Fit RFE to the training data
# rfe.fit(X_train, y_train)
# 
# # Get the selected features
# selected_features = X.columns[rfe.support_]
# 
# # Train the model on selected features
# model.fit(X_train[selected_features], y_train)
# 
# # Make predictions on the testing set
# y_pred = model.predict(X_test[selected_features])
# 
# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)


Accuracy: 0.9850374064837906


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression  # Import your model here

# Assuming 'df' is your DataFrame with features and target variable
# X should contain the features and Y should contain the target variable
# Replace 'target_column' with the name of your target variable column
Y = df['target_column']
X = df.drop(columns=['target_column'])

# Initialize your model
model = LogisticRegression()  # Initialize your model here

# Define the number of folds (k) for cross-validation
num_folds = 5  # You can change this value as needed

# Initialize KFold with the number of folds and random state for reproducibility
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation
scores = cross_val_score(model, X, Y, cv=kf)

# Print the cross-validation scores for each fold
print("Cross-validation scores:", scores)

# Calculate and print the average cross-validation score
average_score = scores.mean()
print("Average cross-validation score:", average_score)
